## 🧪 ChaCha20 – Using `cryptography` (IETF standard)

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.backends import default_backend
import os

# Key and nonce (fixed nonce length)
key = os.urandom(32)             # 256-bit key
nonce = os.urandom(16)           # 128-bit nonce (required by this implementation)
plaintext = b"Hello from ChaCha20 stream cipher!"

# Encrypt
algorithm = algorithms.ChaCha20(key, nonce)
cipher = Cipher(algorithm, mode=None, backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext)

# Decrypt
decryptor = cipher.decryptor()
recovered = decryptor.update(ciphertext)

print("Ciphertext:", ciphertext.hex())
print("Recovered:", recovered)


# Helper: convert bytes to continuous bit string
def to_bits(data: bytes) -> str:
    return ''.join(f'{byte:08b}' for byte in data)

# Recover the keystream: ciphertext ⊕ plaintext
keystream = bytes(c ^ p for c, p in zip(ciphertext, plaintext))

print("Plaintext  (bits):", to_bits(plaintext))
print("Ciphertext (bits):", to_bits(ciphertext))
print("Keystream  (bits):", to_bits(keystream))
print("Recovered  (bits):", to_bits(recovered))

Ciphertext: 705ef5c368bf0166b834495adbb5b96aacad9835b57d6d8f6d9ad1affd11c3bbaa5c
Recovered: b'Hello from ChaCha20 stream cipher!'
Plaintext  (bits): 01001000011001010110110001101100011011110010000001100110011100100110111101101101001000000100001101101000011000010100001101101000011000010011001000110000001000000111001101110100011100100110010101100001011011010010000001100011011010010111000001101000011001010111001000100001
Ciphertext (bits): 01110000010111101111010111000011011010001011111100000001011001101011100000110100010010010101101011011011101101011011100101101010101011001010110110011000001101011011010101111101011011011000111101101101100110101101000110101111111111010001000111000011101110111010101001011100
Keystream  (bits): 001110000011101110011001101011110000011110011111011001110001010011010111010110010110100100011001101100111101010011111010000000101100110110011111101010000001010111000110000010010001111111101010000011001111011111110001110011001001010001100001101010111101111011011000011

## 🧪 Salsa20 – Using `pycryptodome`

In [3]:
from Crypto.Cipher import Salsa20
import os

# Key and nonce
key = os.urandom(32)              # 256-bit key
nonce = os.urandom(8)             # 64-bit nonce
plaintext = b"Hello from Salsa20 stream cipher!"

# Encrypt
cipher = Salsa20.new(key=key, nonce=nonce)
ciphertext = cipher.encrypt(plaintext)

# Decrypt
cipher_dec = Salsa20.new(key=key, nonce=nonce)
recovered = cipher_dec.decrypt(ciphertext)

print("Ciphertext:", ciphertext.hex())
print("Recovered:", recovered)

# Recover the keystream: ciphertext ⊕ plaintext
keystream = bytes(c ^ p for c, p in zip(ciphertext, plaintext))

print("Plaintext  (bits):", to_bits(plaintext))
print("Ciphertext (bits):", to_bits(ciphertext))
print("Keystream  (bits):", to_bits(keystream))
print("Recovered  (bits):", to_bits(recovered))

Ciphertext: b8f77db78717d3020f312689337dbf30339bb4e97b49b77e0c66ae69c933f13e07
Recovered: b'Hello from Salsa20 stream cipher!'
Plaintext  (bits): 010010000110010101101100011011000110111100100000011001100111001001101111011011010010000001010011011000010110110001110011011000010011001000110000001000000111001101110100011100100110010101100001011011010010000001100011011010010111000001101000011001010111001000100001
Ciphertext (bits): 101110001111011101111101101101111000011100010111110100110000001000001111001100010010011010001001001100110111110110111111001100000011001110011011101101001110100101111011010010011011011101111110000011000110011010101110011010011100100100110011111100010011111000000111
Keystream  (bits): 111100001001001000010001110110111110100000110111101101010111000001100000010111000000011011011010010100100001000111001100010100010000000110101011100101001001101000001111001110111101001000011111011000010100011011001101000000001011100101011011100101000100110000100110
Recovered  (bits): 01

## 🧪 Trivium – Toy Python implementation (educational use only)

In [4]:
def trivium_init(key, iv):
    # Trivium uses 80-bit key and IV
    assert len(key) == 10 and len(iv) == 10
    key_bits = sum([[int(b) >> i & 1 for i in range(8)] for b in key], [])
    iv_bits  = sum([[int(b) >> i & 1 for i in range(8)] for b in iv], [])

    state = [0] * 288
    state[:80] = key_bits
    state[93:173] = iv_bits
    state[285:] = [1, 1, 1]

    for _ in range(4 * 288):  # Warm-up phase
        t1 = state[65] ^ (state[90] & state[91]) ^ state[92] ^ state[170]
        t2 = state[161] ^ (state[174] & state[175]) ^ state[176] ^ state[263]
        t3 = state[242] ^ (state[285] & state[286]) ^ state[287] ^ state[68]
        state = [t3] + state[:92] + [t1] + state[93:176] + [t2] + state[177:287]
    return state

def trivium_keystream(state, length):
    stream = []
    for _ in range(length * 8):
        t1 = state[65] ^ state[92]
        t2 = state[161] ^ state[176]
        t3 = state[242] ^ state[287]
        z = t1 ^ t2 ^ t3
        stream.append(z)
        t1 = t1 ^ (state[90] & state[91]) ^ state[170]
        t2 = t2 ^ (state[174] & state[175]) ^ state[263]
        t3 = t3 ^ (state[285] & state[286]) ^ state[68]
        state = [t3] + state[:92] + [t1] + state[93:176] + [t2] + state[177:287]
    return stream

def xor_bytes(data, stream_bits):
    return bytes(b ^ sum([bit << i for i, bit in enumerate(stream_bits[n*8:n*8+8])])
                 for n, b in enumerate(data))

# Key, IV (10 bytes each = 80 bits)
key = b'1234567890'
iv  = b'abcdefghij'
plaintext = b"Trivium stream cipher example!"

# Initialize and encrypt
state = trivium_init(key, iv)
stream_bits = trivium_keystream(state.copy(), len(plaintext))
ciphertext = xor_bytes(plaintext, stream_bits)

# Decrypt
state = trivium_init(key, iv)
stream_bits = trivium_keystream(state.copy(), len(ciphertext))
recovered = xor_bytes(ciphertext, stream_bits)

print("Ciphertext:", ciphertext.hex())
print("Recovered:", recovered)

# Recover the keystream: ciphertext ⊕ plaintext
keystream = bytes(c ^ p for c, p in zip(ciphertext, plaintext))

print("Plaintext  (bits):", to_bits(plaintext))
print("Ciphertext (bits):", to_bits(ciphertext))
print("Keystream  (bits):", to_bits(keystream))
print("Recovered  (bits):", to_bits(recovered))

Ciphertext: 03ec5f853e5fded4abedb8ec9b10d5c217e640eae4df167a453367d64eea
Recovered: b'Trivium stream cipher example!'
Plaintext  (bits): 010101000111001001101001011101100110100101110101011011010010000001110011011101000111001001100101011000010110110100100000011000110110100101110000011010000110010101110010001000000110010101111000011000010110110101110000011011000110010100100001
Ciphertext (bits): 000000111110110001011111100001010011111001011111110111101101010010101011111011011011100011101100100110110001000011010101110000100001011111100110010000001110101011100100110111110001011001111010010001010011001101100111110101100100111011101010
Keystream  (bits): 010101111001111000110110111100110101011100101010101100111111010011011000100110011100101010001001111110100111110111110101101000010111111010010110001010001000111110010110111111110111001100000010001001000101111000010111101110100010101111001011
Recovered  (bits): 01010100011100100110100101110110011010010111010101101101001000000111001101110100011